# Teach a Machine Predict Heart Disease using Data

### Load Dataset

In [ ]:
import pandas as pd

dataset = pd.read_csv("datasets/heart.csv")
data = dataset.copy()

### Seperate input Features and output(labels)

In [ ]:
X = data.drop(["HeartDisease"], axis=1)
y = data["HeartDisease"]

# Define the Preprocessing
numerical = ["Age","RestingBP","Cholesterol","RestingBP","MaxHR","Oldpeak"]
categorical = ["Sex","ChestPainType","RestingECG","ExerciseAngina","ST_Slope","FastingBS"]

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

preprocessor = ColumnTransformer([
    ("num", StandardScaler(), numerical),
    ("cat", OneHotEncoder(handle_unknown="ignore"), categorical)
])

### Split the dataset into Training and Validation sets

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(
    X,y,test_size=0.2,stratify=y,random_state=42
)

### Preprocess data (Scale numbers and encode Categorical data)

In [ ]:
X_train_prep = preprocessor.fit_transform(X_train)
X_test_prep = preprocessor.fit_transform(X_test)

### Build The Neural Network

In [ ]:
# Build Model With Hyperparameters

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.optimizers import Adam

def buid_model(hp):
    model = Sequencial()
    model = model.add(Dense(
        hp.Int("units_1", min_value=32,max_value=128,step=16),
        activation="relu",
        input_shape=(X_test_prep.shape[1],)
    ))
    model.add(Dropout(hp.Float("dropout_1", min_value=0.2,max_value=0.5, step=0.1)))

    model.add(Dense(
        hp.Int("units_2", min_value=16, max_value=64,step=16),
        activation="relu"
    ))

    model.add(Dropout(hp.Float("dropout_2", min_value=0.2, max_value=0.5, step=0.1)))
    model.add(Dense(1, activation="sigmoid"))
    
    model.compile(
        optimizer = Adam(hp.Float("lr", min_value=le-2,sampling="log")),
        loss = "binary_crossentropy",
        matrics = ["accuracy"]
    )

    return model

# Search Tuner
import keras_tuner as kt

tuner = kt.RandomSearch(
    build_model,
    objective="val_accuracy",
    max_trials=10,
    executions_per_trial=1,
    overwrite=True,
    directory="tuner_logs",
    project_name="heart_disease_nn"
)

from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)

# Start tuning
tuner.search(
    X_train_prep, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

# Get best model
best_model = tuner.get_best_models(num_models=1)[0]

# Evaluate
loss, accuracy = best_model.evaluate(X_test_prep, y_test)
y_pred = (best_model.predict(X_test_prep) > 0.5).astype(int)

print("\n Best Accuracy:", round(accuracy * 100, 2), "%")
print("\n Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

# Save model and preprocessor
best_model.save("nn_model.keras")
joblib.dump(preprocessor, "preprocessor.pkl")

print("\nSaved best model as 'nn_model.keras' and preprocessor as 'preprocessor.pkl'")
    